In [10]:
import warnings
warnings.filterwarnings('ignore')
from SPARQLWrapper import SPARQLWrapper, JSON, CSV
import pandas as pd
from pandas.io.json import json_normalize
from itertools import compress
import re
import math
import requests
from rdflib import Graph, Namespace, URIRef, BNode,Literal
from rdflib.namespace import RDF, FOAF, RDFS, XSD, OWL
import rdflib
import rdfextras
import json 
import os
import os.path
from os import path
from haversine import haversine, Unit
import numpy as np


In [11]:
class query_maker:
    #mettere nella stessa cartella di graph creator
    def __init__ (self,filename='graph_dbpedia.ttl',
                 filepath='graph_creator.py'):
        self.filename=filename
        self.filepath=filepath
        if path.exists(self.filename):
            rdfextras.registerplugins()
            g=rdflib.Graph()
            g.parse(self.filename,format='turtle')
            self.g=g
        else:
            self.update_db()
    #aggiorna il grafo rdf accedendo da dbpedia con il file graph_creator.py
    def update_db(self):
        exec(open(self.filepath).read())
        rdfextras.registerplugins()
        g=rdflib.Graph()
        g.parse(self.filename,format='turtle')
        self.g=g
    #query generiche per parsare il grafo in df
    def table_query(self,query,initNs=None,initBindings=None,format='json'):
        self.sparql_query=query
        self.initNs=initNs
        self.initBindings=initBindings
        self.format=format
        if(initNs is None):
            self.initNs=dict(
                foaf=Namespace('http://xmlns.com/foaf/0.1/'),
                dbp=Namespace("http://dbpedia.org/property/"),
                dbo=Namespace("http://dbpedia.org/ontology/"),
                ggart=Namespace("https://ggart.com/ontology/"),
                xsd=Namespace("http://www.w3.org/2001/XMLSchema#"))
            
        result_table = json.loads(self.g.query(self.sparql_query,initNs=self.initNs,
                                                            initBindings=self.initBindings).serialize(format=self.format))
        result_table = json_normalize(result_table["results"]["bindings"])
        return(self.formatter(result_table=result_table))
#     formatta il dataframe prendendo solo i valori
    def formatter(self,result_table):
            Nomi=list(compress(result_table.columns, [".value" in i for i in result_table.columns]))
            table = result_table[Nomi]
            table.columns=[re.sub('.value','', x) for x in table.columns]
            return(table)
    #calcola la geo distance tra due cordinate
    def distance_calculator(self,cord1,cord2):
        return haversine(cord1,cord2)
    #costruisce il vettore distanze
    def dist_calc(self,table,cord):
        Tuple_Cord=[tuple(x) for x in table[['lat','long']].apply(pd.to_numeric).values]
        return([self.distance_calculator(x,cord) for x in Tuple_Cord])
    #trova la risorsa più vicino alle cordinate fornite   
    def nearest(self,table,lat,long):
        cord=(lat,long)
        dist=self.dist_calc(table=table,cord=cord)
        index_min = dist.index(np.nanmin(dist))
        return(table.iloc[index_min])
    #trova le risorse comprese nel raggio pari a radius (in km)
    def radius(self,table,lat,long,radius=10):
        cord=(lat,long)
        dist=self.dist_calc(table=table,cord=cord)
        return(table[pd.Series([x<radius for x in dist])])
    #fa matching con il nome di un'opera d'arte e da tutte le sue informazioni (es: 'Mona Lisa')
    def string_finder(self,name):
        self.name=name
        sparql_query="""
        SELECT DISTINCT ?arte ?artfind ?artewiki ?city ?cityUri ?city_lat ?city_long ?image 
        ?lat ?long ?museum ?museumlabel ?museumwiki  ?sommario ?sommario_museum
        WHERE{
          ?arte foaf:name ?artfind.
          OPTIONAL{
              ?arte foaf:isPrimaryTopicOf ?artewiki.
          }
         OPTIONAL{
              ?arte dbp:city ?cityUri.
              OPTIONAL{
                   ?cityUri rdfs:label ?city.
              }
              OPTIONAL{
                 ?cityUri dbp:latitude ?city_lat;
                          dbp:longitude ?city_long.
              }
         }
          OPTIONAL{
              ?arte dbo:museum ?museum.
              OPTIONAL{
                  ?museum rdfs:label ?museumlabel. 
              }
              OPTIONAL{
                  ?museum foaf:isPrimaryTopicOf ?museumwiki.
              }
              OPTIONAL{
                  ?museum dbo:abstract ?sommario_museum.
              }
          }
          OPTIONAL{
              ?arte dbp:latitude ?lat;
                    dbp:longitude ?long.
          }
         OPTIONAL{
             ?arte foaf:depiction ?image.
         }
          OPTIONAL{
              ?arte dbo:abstract ?sommario.
          }
        }
        """
        initBindings={'artfind':Literal(self.name)}
        self.artfind=self.table_query(query=sparql_query,initBindings=initBindings)
    #trova tutte le opere d'arte in un certo museo in base al suo nome    
    def art_in_museum_finder(self,museum_name):
        self.museum_name=museum_name
        sparql_query="""
        SELECT DISTINCT ?arte ?name ?artewiki ?city ?cityUri ?city_lat ?city_long ?image 
        ?lat ?long ?museum ?museumlabel ?museumwiki  ?sommario
        WHERE{
          ?arte rdf:type ggart:arte_museum;
                foaf:name ?name;
                dbo:museum ?museum.
          ?museum rdfs:label ?museumfind.
          OPTIONAL{
              ?arte foaf:isPrimaryTopicOf ?artewiki.
          }
         OPTIONAL{
              ?arte dbp:city ?cityUri.
              OPTIONAL{
                   ?cityUri rdfs:label ?city.
              }
              OPTIONAL{
                 ?cityUri dbp:latitude ?city_lat;
                          dbp:longitude ?city_long.
              }
         }
          OPTIONAL{
              ?arte dbp:latitude ?lat;
                    dbp:longitude ?long.
          }
          OPTIONAL{
              ?arte dbo:abstract ?sommario.
          }
         OPTIONAL{
             ?arte foaf:depiction ?image.
         }
        }
        """
        initBindings={'museumfind':Literal(self.museum_name)}
        self.artfind=self.table_query(query=sparql_query,initBindings=initBindings)
    #definisce i musei come opere d'arte. Usato per fare in modo di non avere 100 output sulla mappa nello stesso luogo
    #di opere d'arte che appartengono allo stesso museo.
    def collect_museum(self): #museum as art for map
        sparql_query="""
            SELECT DISTINCT  ?arte (SAMPLE(?name) as ?name) ?artewiki ?lat ?long (SAMPLE(?city) as ?city) (AVG(?city_lat) as ?city_lat)
            (AVG(?city_long) as ?city_long) ?sommario
            WHERE{
              ?x rdf:type ggart:arte_museum;
                 dbo:museum ?arte;
                 dbp:city ?cityUri.
             OPTIONAL{
                  ?x dbp:city ?cityUri.
                  OPTIONAL{
                       ?cityUri rdfs:label ?city.
                  }
                  OPTIONAL{
                     ?cityUri dbp:latitude ?city_lat;
                              dbp:longitude ?city_long.
                  }
             }
              OPTIONAL{
                  ?arte rdfs:label ?name. 
              }
              OPTIONAL{
                  ?arte foaf:isPrimaryTopicOf ?artewiki.
              }
              OPTIONAL{
                  ?x    dbp:latitude ?lat;
                        dbp:longitude ?long.
              }
              OPTIONAL{
                  ?arte dbo:abstract ?sommario.
              }
            }GROUP BY ?arte ?artewiki
            """
        self.museum=self.table_query(query=sparql_query)
    #trova tutte le opere d'arte non in un museo
    def collect_not_in_museum(self): #art for map withou museum
        sparql_query="""
            SELECT DISTINCT  ?arte (SAMPLE(?name) as ?name) ?artewiki ?sommario ?lat ?long (SAMPLE(?city) as ?city) 
            (AVG(?city_lat) as ?city_lat) (AVG(?city_long) as ?city_long)
            WHERE{
              ?arte rdf:type ggart:arte_not_in_museum;
                    foaf:name ?name.
             OPTIONAL{
                  ?arte dbp:city ?cityUri.
                  OPTIONAL{
                       ?cityUri rdfs:label ?city.
                  }
                  OPTIONAL{
                     ?cityUri dbp:latitude ?city_lat;
                              dbp:longitude ?city_long.
                  }
             }
              OPTIONAL{
                  ?arte dbo:abstract ?sommario.
              }
              OPTIONAL{
                  ?arte foaf:isPrimaryTopicOf ?artewiki.
              }
              OPTIONAL{
                  ?arte    dbp:latitude ?lat;
                           dbp:longitude ?long.
              }
            }GROUP BY ?arte ?artewiki
            """
        self.not_in_museum=self.table_query(query=sparql_query)
    #trova tutto ciò che è stato preso da dbpedia
    def collect_all(self):
        sparql_query="""
            SELECT DISTINCT ?arte ?name ?artewiki ?city  ?city_lat ?city_long ?image ?lat
            ?long  ?museum ?museumlabel ?museumwiki  ?sommario ?sommario_museum
            WHERE{
              ?arte foaf:name ?name.
              OPTIONAL{
                  ?arte foaf:isPrimaryTopicOf ?artewiki.
              }
             OPTIONAL{
                  ?arte dbp:city ?cityUri.
                  OPTIONAL{
                       ?cityUri rdfs:label ?city.
                  }
                  OPTIONAL{
                     ?cityUri dbp:latitude ?city_lat;
                              dbp:longitude ?city_long.
                  }
             }

              OPTIONAL{
                  ?arte dbo:museum ?museum.
                  OPTIONAL{
                      ?museum rdfs:label ?museumlabel. 
                  }
                  OPTIONAL{
                      ?museum foaf:isPrimaryTopicOf ?museumwiki.
                  }
                  OPTIONAL{
                      ?museum dbo:abstract ?sommario_museum.
                  }
              }
              OPTIONAL{
                  ?arte dbp:latitude ?lat;
                        dbp:longitude ?long.
              }
             OPTIONAL{
                 ?arte foaf:depiction ?image.
             }
              OPTIONAL{
                  ?arte dbo:abstract ?sommario.
              }
            }
            """
        self.all=self.table_query(query=sparql_query)
    #colleziona opere d'arte (non in musei) e opere musei per la rappresentazione in una mappa. (Nel caso è possibile risalire
#     alle opere d'arte nel museo) 
    def art_geomapping(self,lat,long):
        cord=(lat,long)
        if not hasattr(self,'museum'):
            self.collect_museum()
        if not hasattr(self,'not_in_museum'):
            self.collect_not_in_museum()
        self.museum.assign(sommario=np.repeat(np.nan, self.museum.shape[0]).tolist())
        self.artmap=self.museum.append(self.not_in_museum,ignore_index=True)

In [12]:
q=query_maker()


Esempi

In [13]:
q.art_geomapping(lat=45.465454,long=9.186516)
q.nearest(table=q.artmap,lat=45.465454,long=9.186516) #opera d'arte più vicina a cefriel: museo che ha circa 10 opere d'arte

arte           http://dbpedia.org/resource/Museo_del_Novecento
artewiki      http://en.wikipedia.org/wiki/Museo_del_Novecento
city                                                     Milan
city_lat                                     45.46666717529297
city_long                                    9.183333396911621
lat                                           45.4636116027832
long                                         9.190278053283691
name                                       Museo del Novecento
sommario     The Museo del Novecento ("museum of the twenti...
Name: 217, dtype: object

In [14]:
q.radius(table=q.artmap,lat=45.465454,long=9.186516,radius=100) #opere d'arte nel raggio di 100km da cefriel.

,arte,artewiki,city,city_lat,city_long,lat,long,name,sommario
13,http://dbpedia.org/resource/Pinacoteca_di_Brera,http://en.wikipedia.org/wiki/Pinacoteca_di_Brera,Milan,45.46666717529297,9.183333396911621,45.47194290161133,9.188055992126465,Pinacoteca di Brera,"The Pinacoteca di Brera (""Brera Art Gallery"") ..."
103,http://dbpedia.org/resource/Santa_Maria_delle_...,http://en.wikipedia.org/wiki/Santa_Maria_delle...,Milan,45.46666717529297,9.183333396911621,45.4658317565918,9.171111106872559,Santa Maria delle Grazie (Milan),"Santa Maria delle Grazie (""Holy Mary of Grace""..."
217,http://dbpedia.org/resource/Museo_del_Novecento,http://en.wikipedia.org/wiki/Museo_del_Novecento,Milan,45.46666717529297,9.183333396911621,45.4636116027832,9.190278053283691,Museo del Novecento,"The Museo del Novecento (""museum of the twenti..."
240,http://dbpedia.org/resource/Sforza_Castle,http://en.wikipedia.org/wiki/Sforza_Castle,Milan,45.46666717529297,9.183333396911621,45.47000122070312,9.178610801696777,Sforza Castle,Il Castello Sforzesco è uno dei principali sim...
1087,"http://dbpedia.org/resource/Needle,_Thread_and...","http://en.wikipedia.org/wiki/Needle,_Thread_an...",Milan,45.46666717529297,9.183333396911621,45.46812057495117,9.176630973815918,"Needle, Thread and Knot",Il template {{Coord}} ha riscontrato degli err...
1399,http://dbpedia.org/resource/The_Floating_Piers,http://en.wikipedia.org/wiki/The_Floating_Piers,NaN,0,0,45.69606018066406,10.09674739837646,The Floating Piers,Il template {{Coord}} ha riscontrato degli err...
1619,http://dbpedia.org/resource/Life_Electric,http://en.wikipedia.org/wiki/Life_Electric,Como,45.81666564941406,9.083333015441895,45.81538772583008,9.080249786376953,Life Electric,Il template {{Coord}} ha riscontrato degli err...


In [15]:
q.string_finder(name='The Starry Night') #trova opere d'arte chiamate Mona Lisa
q.artfind

,arte,artewiki,artfind,image,lat,long,museum,museumlabel,museumwiki,sommario,sommario_museum
0,http://dbpedia.org/resource/The_Starry_Night,http://en.wikipedia.org/wiki/The_Starry_Night,The Starry Night,http://en.wikipedia.org/wiki/Special:FilePath/...,40.76148223876953,-73.9776611328125,http://dbpedia.org/resource/Museum_of_Modern_Art,Museum of Modern Art,http://en.wikipedia.org/wiki/Museum_of_Modern_Art,"La Notte stellata è un olio su tela (73,7 cm ×...",(This article is about the museum in New York ...


In [23]:
q.all.iloc[list(compress(range(len(t)), t))]

,arte,artewiki,city,city_lat,city_long,image,lat,long,museum,museumlabel,museumwiki,name,sommario,sommario_museum
2298,http://dbpedia.org/resource/The_Starry_Night,http://en.wikipedia.org/wiki/The_Starry_Night,NaN,NaN,NaN,http://en.wikipedia.org/wiki/Special:FilePath/...,40.76148223876953,-73.9776611328125,http://dbpedia.org/resource/Museum_of_Modern_Art,Museum of Modern Art,http://en.wikipedia.org/wiki/Museum_of_Modern_Art,The Starry Night,"La Notte stellata è un olio su tela (73,7 cm ×...",(This article is about the museum in New York ...


In [22]:
from itertools import compress
t=["The Starry Night" in i for i in q.all['name']]
list(compress(range(len(t)), t))

[2298]

In [18]:
q.collect_all() #colleziona tutto
q.all

,arte,artewiki,city,city_lat,city_long,image,lat,long,museum,museumlabel,museumwiki,name,sommario,sommario_museum
0,http://dbpedia.org/resource/The_Astronomer_(Ve...,http://en.wikipedia.org/wiki/The_Astronomer_(V...,Paris,48.8567008972168,2.350800037384033,http://commons.wikimedia.org/wiki/Special:File...,NaN,NaN,http://dbpedia.org/resource/Musée_du_Louvre,Musée du Louvre,http://en.wikipedia.org/wiki/Musée_du_Louvre,The Astronomer,Astronomo è un dipinto a olio su tela (50x45 c...,NaN
1,"http://dbpedia.org/resource/Bottle,_Glass,_Fork","http://en.wikipedia.org/wiki/Bottle,_Glass,_Fork",NaN,NaN,NaN,NaN,41.50888824462891,-81.61166381835938,http://dbpedia.org/resource/Cleveland_Museum_o...,Cleveland Museum of Art,http://en.wikipedia.org/wiki/Cleveland_Museum_...,"Bottle, Glass, Fork","Bottle, Glass, Fork (otherwise known as Boutei...",The Cleveland Museum of Art (CMA) is an art mu...
2,http://dbpedia.org/resource/Fontaine_du_Palmier,http://en.wikipedia.org/wiki/Fontaine_du_Palmier,NaN,NaN,NaN,http://commons.wikimedia.org/wiki/Special:File...,48.85749816894531,2.347273707389832,NaN,NaN,NaN,Fountain of the Palm,The Fontaine du Palmier (1806-1808) or Fontain...,NaN
3,http://dbpedia.org/resource/The_Farmers'_Lunch,http://en.wikipedia.org/wiki/The_Farmers'_Lunch,Budapest,47.49250030517578,19.05138969421387,http://commons.wikimedia.org/wiki/Special:File...,47.51610946655273,19.07638931274414,http://dbpedia.org/resource/Museum_of_Fine_Art...,Museum of Fine Arts (Budapest),http://en.wikipedia.org/wiki/Museum_of_Fine_Ar...,The Farmers' Lunch,Il pranzo degli agricoltori è un dipinto a oli...,The Museum of Fine Arts (Hungarian: Szépművész...
4,http://dbpedia.org/resource/The_Third_of_May_1808,http://en.wikipedia.org/wiki/The_Third_of_May_...,Madrid,40.38333511352539,-3.716666698455811,http://en.wikipedia.org/wiki/Special:FilePath/...,40.41380500793457,-3.692456007003784,http://dbpedia.org/resource/Museo_del_Prado,Museo del Prado,http://en.wikipedia.org/wiki/Museo_del_Prado,The Third of May 1808,Il 3 maggio 1808 (anche conosciuto come El tre...,The Prado Museum (Spanish pronunciation: [muˈs...
5,http://dbpedia.org/resource/Calvin_Fletcher_(b...,http://en.wikipedia.org/wiki/Calvin_Fletcher_(...,NaN,NaN,NaN,NaN,NaN,NaN,http://dbpedia.org/resource/Indiana_Statehouse,Indiana Statehouse,http://en.wikipedia.org/wiki/Indiana_Statehouse,Calvin Fletcher,Calvin Fletcher is a public artwork by an unkn...,NaN
6,http://dbpedia.org/resource/I_Saw_the_Figure_5...,http://en.wikipedia.org/wiki/I_Saw_the_Figure_...,United States of America,NaN,NaN,http://commons.wikimedia.org/wiki/Special:File...,40.779178619384766,-73.9633903503418,http://dbpedia.org/resource/Metropolitan_Museu...,Metropolitan Museum of Art,http://en.wikipedia.org/wiki/Metropolitan_Muse...,I Saw the Figure 5 in Gold,"I Saw the Figure 5 in Gold, also known as The ...","The Metropolitan Museum of Art, colloquially ""..."
7,http://dbpedia.org/resource/I_Saw_the_Figure_5...,http://en.wikipedia.org/wiki/I_Saw_the_Figure_...,New York City,40.71269989013672,-74.00589752197266,http://commons.wikimedia.org/wiki/Special:File...,40.779178619384766,-73.9633903503418,http://dbpedia.org/resource/Metropolitan_Museu...,Metropolitan Museum of Art,http://en.wikipedia.org/wiki/Metropolitan_Muse...,I Saw the Figure 5 in Gold,"I Saw the Figure 5 in Gold, also known as The ...","The Metropolitan Museum of Art, colloquially ""..."
8,http://dbpedia.org/resource/Sky_Landscape_I,http://en.wikipedia.org/wiki/Sky_Landscape_I,NaN,NaN,NaN,NaN,47.61639022827148,-122.3552780151367,http://dbpedia.org/resource/Olympic_Sculpture_...,Olympic Sculpture Park,http://en.wikipedia.org/wiki/Olympic_Sculpture...,Sky Landscape I,Sky Landscape I is an outdoor 1983 painted alu...,"The Olympic Sculpture Park, created and operat..."
9,http://dbpedia.org/resource/Early_works_of_Vin...,http://en.wikipedia.org/wiki/Early_works_of_Vi...,NaN,NaN,NaN,http://commons.wikimedia.org/wiki/Special:File...,52.09555435180664,5.816944599151611,http://dbp

In [19]:
q.collect_museum() #trova tutti i musei
q.museum

,arte,artewiki,city,city_lat,city_long,lat,long,name,sommario
0,http://dbpedia.org/resource/Indianapolis_Art_C...,http://en.wikipedia.org/wiki/Indianapolis_Art_...,Indiana,39.5536048412323,-87.89987659454346,39.8778076171875,-86.14363861083984,Indianapolis Art Center,The Indianapolis Art Center is an art center l...
1,http://dbpedia.org/resource/Musée_des_Beaux-Ar...,http://en.wikipedia.org/wiki/Musée_des_Beaux-A...,Besançon,47.24309921264648,6.021900177001953,47.24020004272461,6.023099899291992,Musée des Beaux-Arts et d'archéologie de Besançon,The Musée des Beaux-Arts et d'archéologie (Mus...
2,"http://dbpedia.org/resource/Sutton,_St_Helens","http://en.wikipedia.org/wiki/Sutton,_St_Helens","St Helens, Merseyside",53.4541015625,-2.746099948883057,53.43600082397461,-2.703999996185303,"Sutton, St Helens","Sutton is an area of St Helens, Merseyside, an..."
3,http://dbpedia.org/resource/Museo_del_Prado,http://en.wikipedia.org/wiki/Museo_del_Prado,Madrid,40.569050675731596,-3.658093251414219,40.41380500793457,-3.692456007003784,Museo del Prado,The Prado Museum (Spanish pronunciation: [muˈs...
4,http://dbpedia.org/resource/Louvre,http://en.wikipedia.org/wiki/Louvre,Paris,48.635005593299866,2.839676309376955,48.86072540283203,2.336993932723999,Louvre,The Louvre or the Louvre Museum (French: Musée...
5,http://dbpedia.org/resource/Museum_of_Fine_Art...,http://en.wikipedia.org/wiki/Museum_of_Fine_Ar...,Budapest,47.49250030517578,19.05138969421387,47.51610946655273,19.07638931274414,Museum of Fine Arts (Budapest),The Museum of Fine Arts (Hungarian: Szépművész...
6,http://dbpedia.org/resource/National_Gallery,http://en.wikipedia.org/wiki/National_Gallery,London,51.660995819989374,-0.25374999784809693,51.50859832763672,-0.128299996256828,National Gallery,The National Gallery is an art museum in Trafa...
7,http://dbpedia.org/resource/Fundació_Joan_Miró,http://en.wikipedia.org/wiki/Fundació_Joan_Miró,Barcelona,41.38333511352539,2.183333396911621,41.36861038208008,2.160000085830688,Fundació Joan Miró,"The Fundació Joan Miró, Centre d'Estudis d'Art..."
8,http://dbpedia.org/resource/National_Gallery_o...,http://en.wikipedia.org/wiki/National_Gallery_...,"Washington, D.C.",41.8403726264612,-58.16066054443815,38.89147186279297,-77.02001190185547,National Gallery of Art,"The National Gallery of Art, and its attached ..."
9,http://dbpedia.org/resource/Städelsches_Kunsti...,http://en.wikipedia.org/wiki/Städelsches_Kunst...,Frankfurt,50.11666488647461,8.683333396911621,NaN,NaN,Städelsches Kunstinstitut und Städtische Galerie,NaN


In [20]:
q.collect_not_in_museum() #trova tutte opere d'arte non in musei
q.not_in_museum

,arte,artewiki,city,city_lat,city_long,lat,long,name,sommario
0,http://dbpedia.org/resource/Bagnacavallo_Madonna,http://en.wikipedia.org/wiki/Bagnacavallo_Madonna,Traversetolo,44.63333511352539,10.38333320617676,NaN,NaN,Bagnacavallo Madonna,La Madonna di Bagnacavallo è un dipinto a olio...
1,http://dbpedia.org/resource/Golden_(sculpture),http://en.wikipedia.org/wiki/Golden_(sculpture),"Tunstall, Staffordshire",53.05830001831055,-2.211400032043457,NaN,NaN,Golden,Golden is a modern art sculpture due to be ins...
2,http://dbpedia.org/resource/1985_in_fine_arts_...,http://en.wikipedia.org/wiki/1985_in_fine_arts...,Moscow,55.75166702270508,37.61777877807617,NaN,NaN,Monument to Lenin,The year 1985 was marked by many events that l...
3,http://dbpedia.org/resource/Peace_Chant,http://en.wikipedia.org/wiki/Peace_Chant,NaN,0,0,45.51535034179688,-122.6831893920898,Peace Chant,Peace Chant is an outdoor 1984 granite memoria...
4,http://dbpedia.org/resource/Untitled_(Patel),http://en.wikipedia.org/wiki/Untitled_(Patel),NaN,0,0,42.99087905883789,-87.94788360595703,Untitled,Untitled is a public art work by sculptor Nare...
5,http://dbpedia.org/resource/Fallen_Firefighter...,http://en.wikipedia.org/wiki/Fallen_Firefighte...,Seattle,47.60972213745117,-122.3330535888672,47.60018157958984,-122.3329162597656,Fallen Firefighters Memorial,Fallen Firefighters Memorial is a bronze sculp...
6,http://dbpedia.org/resource/Chief_Seattle_(scu...,http://en.wikipedia.org/wiki/Chief_Seattle_(sc...,NaN,0,0,47.61838150024414,-122.3474197387695,Chief Seattle,Chief Seattle is an outdoor life-size sculptur...
7,http://dbpedia.org/resource/La_Mano_de_Punta_d...,http://en.wikipedia.org/wiki/La_Mano_de_Punta_...,Punta del Este,-34.96666717529297,-54.95000076293945,-34.95786285400391,-54.93717956542969,La Mano,La Mano (The Hand) is a sculpture in Punta del...
8,http://dbpedia.org/resource/Memorial_Fountain_...,http://en.wikipedia.org/wiki/Memorial_Fountain...,NaN,0,0,39.9375,-77.66194152832031,Memorial Fountain and Statue,Memorial Fountain and Statue are a historic fo...
9,http://dbpedia.org/resource/Monument_of_Zalongo,http://en.wikipedia.org/wiki/Monument_of_Zalongo,NaN,0,0,39.14849853515625,20.68429946899414,Monument of Zalongo,The Monument of Zalongo is a 1961 monumental s...


In [21]:
q.art_in_museum_finder('Museo de Antioquia') #trova opere d'arte in musei
q.artfind

,arte,artewiki,city,cityUri,city_lat,city_long,image,museum,name,sommario
0,http://dbpedia.org/resource/Horizontes,http://en.wikipedia.org/wiki/Horizontes,Medellín,http://dbpedia.org/resource/Medellín,6.230833530426025,-75.5905532836914,http://commons.wikimedia.org/wiki/Special:File...,http://dbpedia.org/resource/Museo_de_Antioquia,Horizontes,Horizons (Spanish: Horizontes) is a painting b...
